In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, clear_output
from xgboost import XGBClassifier

In [ ]:
dataset= pd.read_csv("stroke prediction.csv")
dataset.head()

In [ ]:
dataset.drop(columns=["id","Residence_type"],inplace=True)

In [ ]:
dataset["gender"]=dataset["gender"].astype('category')
dataset["ever_married"]=dataset["ever_married"].astype('category')
# dataset["Residence_type"]=dataset["Residence_type"].astype('category')
dataset["work_type"]=dataset["work_type"].astype('category')
dataset["smoking_status"]=dataset.smoking_status.astype("category")

In [ ]:
dataset_imputed=dataset.copy()

In [ ]:
dataset_imputed.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

dataset_imputed.gender=encoder.fit_transform(dataset_imputed.gender)

dataset_imputed.ever_married=encoder.fit_transform(dataset_imputed.ever_married)

dataset_imputed.work_type=encoder.fit_transform(dataset_imputed.work_type)

# dataset_imputed.Residence_type=encoder.fit_transform(dataset_imputed.Residence_type)

dataset_imputed.smoking_status=dataset_imputed.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

In [ ]:
X=dataset_imputed.drop(columns="stroke",axis=1)
y=dataset_imputed["stroke"]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.33,random_state=88,stratify=y)

In [186]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()

X_trimputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)

X_teimputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [187]:
X_trimputed["smoking_status"]=X_trimputed["smoking_status"].apply(round)
X_trimputed["smoking_status"].value_counts()

# X_trimputed["smoking_status"]=X_trimputed["smoking_status"].astype('category')
X_trimputed["work_type"]=X_trimputed["work_type"].apply(int)
# X_trimputed["work_type"]=X_trimputed["work_type"].astype('category')

In [188]:
X_teimputed["smoking_status"]=X_teimputed["smoking_status"].apply(round)
X_teimputed["smoking_status"].value_counts()

# X_teimputed["smoking_status"]=X_teimputed["smoking_status"].astype('category')
X_teimputed["work_type"]=X_teimputed["work_type"].apply(int)
# X_teimputed["work_type"]=X_teimputed["work_type"].astype('category')

In [189]:
# X_trimputed["gender"]=X_trimputed["gender"].astype("category")
X_trimputed[["hypertension","heart_disease","ever_married"]]=X_trimputed[["hypertension","heart_disease","ever_married"]].astype("int")

# X_teimputed["gender"]=X_teimputed["gender"].astype("category")
X_teimputed[["hypertension","heart_disease","ever_married"]]=X_teimputed[["hypertension","heart_disease","ever_married"]].astype("int")

In [49]:
X_trimputed=pd.get_dummies(X_trimputed,drop_first=True)
X_teimputed=pd.get_dummies(X_teimputed,drop_first=True)

In [50]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

scaler.fit(X_trimputed)

X_trimputed=pd.DataFrame(scaler.transform(X_trimputed),columns=X_trimputed.columns)

X_teimputed=pd.DataFrame(scaler.transform(X_teimputed),columns=X_teimputed.columns)

In [53]:
from sklearn.metrics import classification_report,roc_auc_score,f1_score

In [54]:
X_trimputed

,age,hypertension,heart_disease,ever_married,avg_glucose_level,bmi,gender_1.0,gender_2.0,work_type_1,work_type_2,work_type_3,work_type_4,smoking_status_1,smoking_status_2
0,0.047852,0.0,0.0,0.0,0.345280,0.080000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.108887,0.0,0.0,0.0,0.150568,0.196571,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.353027,0.0,0.0,1.0,0.227419,0.222857,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.768066,0.0,0.0,1.0,0.125233,0.241143,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.816895,0.0,0.0,1.0,0.065328,0.185143,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34715,0.755859,0.0,0.0,1.0,0.016480,0.217143,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
34716,0.523926,0.0,0.0,0.0,0.168658,0.224000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34717,0.279785,0.0,0.0,0.0,0.184926,0.208000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34718,0.108887,0.0,0.0,0.0,0.402432,0.083429,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# XGBoost

In [250]:
#creating an Gradient boosting instance
clf = XGBClassifier(random_state=40,learning_rate=2.5,subsample=0.7)

In [251]:
#training the model
clf.fit(X_trimputed,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=2.5, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=40, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [252]:
y_train_pred=clf.predict(X_trimputed)
y_test_pred=clf.predict(X_teimputed)

In [253]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     28553
           1       0.04      0.04      0.04       525

    accuracy                           0.96     29078
   macro avg       0.51      0.51      0.51     29078
weighted avg       0.97      0.96      0.96     29078



In [254]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     14064
           1       0.02      0.02      0.02       258

    accuracy                           0.97     14322
   macro avg       0.50      0.50      0.50     14322
weighted avg       0.96      0.97      0.97     14322



## Grid Search

In [263]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [258]:
parameters={'subsample':[0.7,0.8,0.9,1],
            'base_score':[0.2,0.3,0.4,0.5,0.6],
            'gamma':np.arange(0,2.5,0.1),
            'learning_rate':np.arange(0.5,2.5,0.1),
            'max_depth':[2,3,4,5,6,7,8,9,10]}

In [270]:
search=RandomizedSearchCV(XGBClassifier(random_state=40),parameters,scoring='f1',cv=5,n_jobs=-1,verbose=2,n_iter=500)

In [271]:
search.fit(X_trimputed,y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 14.2min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                        'gamma': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.

In [272]:
search.best_score_

0.09581351164886479

In [275]:
search.best_params_

{'subsample': 0.7,
 'max_depth': 9,
 'learning_rate': 2.3,
 'gamma': 2.2,
 'base_score': 0.3}

In [276]:
#creating an Gradient boosting instance
clf = XGBClassifier(random_state=96, max_depth=9, learning_rate=2.3, gamma=2.2, base_score=0.3, n_estimators=200, subsample=0.7)

In [277]:
#training the model
clf.fit(X_trimputed,y_train)

XGBClassifier(base_score=0.3, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=2.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=2.3, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=96, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [278]:
y_train_pred=clf.predict(X_trimputed)
y_test_pred=clf.predict(X_teimputed)

In [279]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     28553
           1       0.19      0.29      0.23       525

    accuracy                           0.96     29078
   macro avg       0.59      0.63      0.61     29078
weighted avg       0.97      0.96      0.97     29078



In [280]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     14064
           1       0.07      0.12      0.09       258

    accuracy                           0.96     14322
   macro avg       0.53      0.55      0.53     14322
weighted avg       0.97      0.96      0.96     14322



## Changing Parameters

In [36]:
#Importing GBDT Classifier 
from sklearn.ensemble import GradientBoostingClassifier

In [49]:
#creating an Gradient boosting instance
clf = GradientBoostingClassifier(random_state=96, 
                                 min_samples_split=100, 
                                 max_depth=10,
                                 n_iter_no_change=5,
                                 tol = 0.001,
                                 validation_fraction=0.2,
                                 subsample=0.8)

In [50]:
#creating an Gradient boosting instance
clf = GradientBoostingClassifier(random_state=40)

In [51]:
#training the model
clf.fit(X_trimputed,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=40, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [52]:
y_train_pred=clf.predict(X_trimputed)
y_test_pred=clf.predict(X_teimputed)

In [53]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     34094
           1       0.78      0.03      0.06       626

    accuracy                           0.98     34720
   macro avg       0.88      0.52      0.53     34720
weighted avg       0.98      0.98      0.97     34720



In [54]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8523
           1       0.00      0.00      0.00       157

    accuracy                           0.98      8680
   macro avg       0.49      0.50      0.50      8680
weighted avg       0.96      0.98      0.97      8680

